In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re

In [2]:
model_name = "eryk-mazus/polka-1.1b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
).to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [14]:
def most_common_element(lst):
    counter = Counter(lst)
    most_common = counter.most_common(1)  # Get the most common element and its count
    return most_common[0][0] if most_common else "0"  # Return the element itself

In [11]:
def generate_one_word_response(chat, prompt, word_set, n = 5):
    d = {}
    for _ in range(n):
        chat.append({"role": "user", "content": prompt})
        inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
        first_param_device = next(model.parameters()).device
        inputs = inputs.to(first_param_device)
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=50,
            temperature=0.2,
            repetition_penalty=1.15,
            top_p=0.95,
            do_sample=True,
            streamer=streamer,
        )
        new_tokens = outputs[0, inputs.size(1):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True)
        print(response)
        for w in response.split():
            if w in word_set:
                d[w] = d.get(w, 0) + 1
    return d
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

In [12]:
# You are a helpful assistant.
system_prompt = "Jesteś pomocnym asystentem i odpowiadasz na pytania na wiedzę"
chat = ([{"role": "system", "content": system_prompt}])


In [13]:
ws = ["jabłko", "pomarańcza", "gruszka", "śliwka", "arbuz", "pomidor"]
generate_one_word_response(chat, "Jaki owoc spadł na głowę Newtona?", ws, n = 5)

<|im_start|> system
Jesteś pomocnym asystentem i odpowiadasz na pytania na wiedzę<|im_end|> 
<|im_start|> user
Jaki owoc spadł na głowę Newtona?<|im_end|> 
<|im_start|> assistant


KeyboardInterrupt: 